You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

# Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: Exploring the Latest Trends in Artificial Intelligence 

Outline:
I. Introduction
- Definition of Artificial Intelligence (AI)
- Importance of AI in various industries
- Overview of the latest trends in AI

II. Key Trends in Artificial Intelligence
A. Machine Learning
- Advancements in deep learning
- Applications in healthcare, finance, and marketing
B. Natural Language Processing (NLP)
- Development of chatbots and virtual assistants
- Language translation and sentiment analysis
C. Computer Vision
- P

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Exploring the Latest Trends in Artificial Intelligence

Artificial Intelligence (AI) has rapidly become a pivotal technology across various industries, revolutionizing the way businesses operate and improving efficiency and decision-making processes. The latest trends in AI showcase groundbreaking advancements in machine learning, natural language processing, and computer vision, offering unprecedented opportunities for innovation and growth.

## Key Trends in Artificial Intelligence
### Machine Learning
Machine learning, a subset of AI, has witnessed significant advancements in deep learning algorithms, leading to remarkable breakthroughs in predictive analytics and data processing. Its applications in healthcare, finance, and marketing have transformed the way organizations analyze data and make informed decisions, driving operational efficiencies and enhancing customer experiences.

### Natural Language Processing (NLP)
The development of NLP technologies has paved the way for the creation of chatbots and virtual assistants that can interact with users in natural language. Furthermore, language translation and sentiment analysis capabilities have improved drastically, enabling businesses to engage with global audiences and gain valuable insights from customer feedback.

### Computer Vision
Advancements in computer vision have revolutionized image recognition and object detection technologies, fueling the growth of applications in autonomous vehicles and surveillance systems. The ability of AI-powered systems to analyze and interpret visual data has opened up new possibilities for enhanced security measures and operational efficiencies in various industries.

## Key Players in Artificial Intelligence
### Tech Giants
Leading tech companies such as Google, Microsoft, and Amazon have heavily invested in AI research and development, driving innovations in machine learning algorithms and AI solutions. Their contributions to the field have set the benchmark for AI excellence and continue to shape the future of technology across industries.

### Startups
Emerging AI startups are disrupting traditional industries with innovative AI algorithms and solutions that address specific business challenges. These startups are driving advancements in AI applications and spearheading the development of cutting-edge technologies that have the potential to transform the way businesses operate and interact with customers.

## Noteworthy News in Artificial Intelligence
### Ethical Concerns
As AI technologies become more prevalent in society, discussions on bias and privacy issues in AI systems have gained prominence. There is a growing call for regulation and transparency in AI development to ensure the ethical use of AI and mitigate potential risks associated with algorithmic decision-making.

### Breakthroughs in Research
Recent achievements in reinforcement learning and neural networks have pushed the boundaries of AI capabilities, opening up new possibilities for future applications in various industries. These breakthroughs have significant implications for the development of AI technologies and underscore the importance of ongoing research and innovation in the field.

## Target Audience Analysis
Tech enthusiasts, professionals, business leaders, students, and researchers interested in artificial intelligence will find value in staying updated on the latest trends and developments in AI. By exploring AI courses or certifications and delving into resources on AI topics, readers can deepen their understanding of AI technologies and their potential impact on society.

## Call to Action
To stay informed on AI news and developments, readers are encouraged to explore resources such as "The State of AI in 2021" by Forbes, "Machine Learning Yearning" by Andrew Ng, and "Ethics in Artificial Intelligence" by MIT Technology Review. By staying engaged with AI trends and advancements, individuals can contribute to the ongoing dialogue on the ethical and practical implications of AI technologies.

By incorporating SEO keywords such as Artificial Intelligence trends, AI developments, machine learning advancements, top AI companies, and ethical concerns in AI, this blog post aims to provide valuable insights for readers interested in the latest trends in Artificial Intelligence. Let's continue to explore the ever-evolving landscape of AI and its impact on society.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Become the googler"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Become the googler.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: Become the Googler: Navigating the Latest Trends in Online Search

Outline:
I. Introduction
- Definition of "becoming the googler"
- Importance of staying updated on search trends
- Preview of key points to be discussed

II. Latest Trends in Online Search
- Voice search technology and its impact on SEO
- Mobile optimization and its role in search rankings
- Local SEO strategies for businesses
- The rise of featured snippets and zero-click searches

III. Key Players in the Search Industry
- Go

In [14]:
Markdown(result)

# Become the Googler: Navigating the Latest Trends in Online Search

In today's digital age, the term "becoming the googler" has taken on a whole new meaning. It's no longer just about using Google as a search engine, but rather about staying ahead of the curve and being knowledgeable about the latest trends in online search. As technology continues to evolve, so do the ways in which we search for information. This blog post will delve into the importance of staying updated on search trends and provide insights into the key players in the search industry, the latest trends in online search, and noteworthy news that every aspiring "googler" should be aware of.

## Latest Trends in Online Search

One of the most significant trends impacting online search is voice search technology. With the rise of virtual assistants like Siri, Alexa, and Google Assistant, optimizing for voice search has become crucial for businesses looking to improve their SEO rankings. Additionally, mobile optimization plays a key role in search rankings, as more and more users are searching for information on their smartphones and tablets. Local SEO strategies are also essential for businesses targeting specific geographic locations, while the prevalence of featured snippets and zero-click searches are changing the way search results are displayed.

## Key Players in the Search Industry

When it comes to online search, Google reigns supreme. With its dominant market share and frequent algorithm updates, staying on top of Google's latest changes is essential for anyone involved in SEO. However, Google is not the only player in the search market. Competitors like Bing, Yahoo, and DuckDuckGo also play a role in shaping the search landscape. Additionally, influential SEO experts and thought leaders provide valuable insights and guidance for navigating the ever-changing world of online search.

## Noteworthy News in the Search World

Keeping up with the latest news and updates in the search industry is crucial for staying ahead of the competition. Recent updates to Google's search algorithms can have a significant impact on website rankings, making it essential to adapt and adjust SEO strategies accordingly. Case studies of successful SEO strategies can provide valuable insights into what works and what doesn't, while emerging trends in search engine marketing offer new opportunities for growth and visibility.

## Target Audience Analysis

This blog post is aimed at small business owners, digital marketers, and entrepreneurs looking to enhance their online visibility and SEO skills. By implementing the latest search trends and strategies discussed in this article, readers can boost their online presence and drive more traffic to their websites. Whether you're a novice or an experienced professional, staying informed and proactive in the world of online search is key to success.

In conclusion, becoming the googler is about more than just using a search engine – it's about understanding and leveraging the latest trends and strategies in online search. By following this comprehensive content plan and staying informed about the key players, trends, and news in the search industry, you can position yourself as an expert in your field and drive success in the digital landscape. Stay ahead of the curve by becoming the googler in your industry and watch your online presence soar.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

In [17]:
topic = "what is the feature about the AI Agent"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on what is the feature about the AI Agent.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now have all the necessary information to create a comprehensive content plan for the blog article on what is the feature about the AI Agent.

Final Answer: 

Content Plan: What is the Feature About the AI Agent

Introduction:
- Brief overview of AI technology and its impact on various industries
- Introduction to AI Agents and their role in enhancing efficiency and productivity

Key Points:
1. Latest Trends in AI Agent Features:
- Natural Language Processing capabilities for improved communication
- Machine learning algorithms for predi